### Univariate

#### ARIMA -- 44.12%

Average MAPE summed: 13.13%  
Average MAPE daily: 44.12%

In [ ]:
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({'quantity': 'sum'})
      .reset_index())

daily_df = daily_df.sort_values('order_date')

results = list()
forecast_results = list()

for pizza in daily_df['pizza_name_id'].unique():

    pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
    pizza_df.set_index('order_date', inplace=True)
    pizza_df = pizza_df.asfreq('D', fill_value=0)

    y = pizza_df['quantity']

    if len(y) < 60:
        continue

    ## d value

    d = 1 if adfuller(y)[1] > 0.05 else 0

    ## Train Test

    train_size = int(len(y) * 0.8)

    y_train = y.iloc[:train_size]
    y_test  = y.iloc[train_size:]

    ## p, q values -- AIC

    p = q = range(0, 3)

    best_aic = np.inf
    best_order = None

    for i, j in itertools.product(p, q):
      try:
          model = SARIMAX(
              y_train,
              order=(i, d, j),
              enforce_stationarity=False,
              enforce_invertibility=False
          )

          model_fit = model.fit(disp=False)

          if model_fit.aic < best_aic:
              best_aic = model_fit.aic
              best_order = (i, d, j)

      except:
          continue

    ## Train

    final_model = SARIMAX(
    y_train,
    order=best_order,
    enforce_stationarity=False,
    enforce_invertibility=False
    )

    final_fit = final_model.fit(disp=False)

    # Test Prediction
    test_pred = final_fit.predict(
      start=len(y_train),
      end=len(y_train) + len(y_test) - 1,
    )

    ## Evaluation

    # Summed MAPE
    actual_sum = y_test.sum()
    predicted_sum = test_pred.sum()

    if actual_sum == 0:
        mape_s = np.nan
    else:
        mape_s = abs((actual_sum - predicted_sum) / actual_sum) * 100

    # Daily MAPE
    mask = y_test.values != 0

    if mask.sum() > 0:
        mape = mean_absolute_percentage_error(
            y_test.values[mask],
            test_pred.values[mask]
        ) * 100
    else:
        mape = np.nan

    ## Forecast

    future_dates = pd.date_range(
      start=y.index[-1] + pd.Timedelta(days=1),
      periods=7,
      freq='D'
    )

    future_forecast = final_fit.forecast(steps=7)
    future_forecast_daily = future_forecast.tail(7)
    future_forecast = future_forecast_daily.sum().round().astype(int)


    results.append({
      'pizza_name_id': pizza,
      'p_d_q': best_order,
      'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
  })

    forecast_results.append({
        'pizza_name_id': pizza,
        'Forecast Quantity': future_forecast,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

results_df = pd.DataFrame(results).sort_values('MAPE')
forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

print(results_df.head())
forecast_df

Average MAPE summed: 13.13%
Average MAPE daily: 44.12%
     pizza_name_id      p_d_q  MAPE_s   MAPE
26  spinach_supr_s  (1, 0, 2)   10.95  19.34
34   pep_msh_pep_l  (1, 0, 2)   16.08  21.17
72     the_greek_l  (2, 0, 2)   48.67  21.36
82  spinach_supr_l  (2, 0, 2)   47.97  22.66
47     ckn_pesto_l  (1, 0, 2)    8.05  22.96


,pizza_name_id,Forecast Quantity,MAPE_s,MAPE
0,bbq_ckn_l,16,12.27,51.57
1,calabrese_m,10,21.98,42.98
2,thai_ckn_m,9,3.61,35.59
3,mediterraneo_m,5,32.54,49.73
4,prsc_argla_m,12,12.35,41.07
...,...,...,...,...
86,soppressata_s,5,3.28,35.02
87,mediterraneo_s,6,8.27,38.11
88,ital_veggie_l,3,13.97,59.26
89,calabrese_s,2,15.02,77.15


#### SARIMA -- 43.93%

Average MAPE summed: 13.00%  
Average MAPE daily: 43.93%

In [ ]:
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum'})
      .reset_index()
)

daily_df = daily_df.sort_values('order_date')

results = list()
forecast_results = list()

for pizza in daily_df['pizza_name_id'].unique():

    pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
    pizza_df.set_index('order_date', inplace=True)
    pizza_df = pizza_df.asfreq('D', fill_value=0)

    y = pizza_df['quantity']

    if len(y) < 60:
        continue

    ## d value

    d = 1 if adfuller(y)[1] > 0.05 else 0

    ## D value

    D = 1 if adfuller(y.diff(7).dropna())[1] > 0.05 else 0

    s = 7

    ## Train Test

    train_size = int(len(y) * 0.8)

    y_train = y.iloc[:train_size]
    y_test  = y.iloc[train_size:]

    ## p, q, P, Q values -- AIC

    p = q = range(0, 3)
    P = Q = range(0, 2)

    best_aic = np.inf
    best_order = None
    best_seasonal = None

    for i, j, m, n in itertools.product(p, q, P, Q):
      try:
          model = SARIMAX(
              y_train,
              order=(i, d, j),
              seasonal_order=(m, D, n, s),
              enforce_stationarity=False,
              enforce_invertibility=False
          )

          model_fit = model.fit(disp=False)

          if model_fit.aic < best_aic:
              best_aic = model_fit.aic
              best_order = (i, d, j)
              best_seasonal = (m, D, n, s)

      except:
          continue

    ## Train

    final_model = SARIMAX(
    y_train,
    order=best_order,
    seasonal_order=best_seasonal,
    enforce_stationarity=False,
    enforce_invertibility=False
    )

    final_fit = final_model.fit(disp=False)

    # Test Prediction
    test_pred = final_fit.predict(
      start=len(y_train),
      end=len(y_train) + len(y_test) - 1)

    ## Evaluate

    # Summed MAPE

    actual_sum = y_test.sum()
    predicted_sum = test_pred.sum()

    if actual_sum == 0:
        mape_s = np.nan
    else:
        mape_s = abs((actual_sum - predicted_sum) / actual_sum) * 100

    # Daily MAPE

    mask = y_test.values != 0

    if mask.sum() > 0:
        mape = mean_absolute_percentage_error(
            y_test.values[mask],
            test_pred.values[mask]
        ) * 100
    else:
        mape = np.nan

    ## Forecast

    future_dates = pd.date_range(
      start=y.index[-1] + pd.Timedelta(days=1),
      periods=7,
      freq='D'
    )

    future_forecast = final_fit.forecast(steps=7)
    future_forecast_daily = future_forecast.tail(7)
    future_forecast = future_forecast_daily.sum().round().astype(int)


    results.append({
      'pizza_name_id': pizza,
      'p_d_q': best_order,
      'P_D_Q_s': best_seasonal,
      'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
  })

    forecast_results.append({
        'pizza_name_id': pizza,
        'Forecast Quantity': future_forecast,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

results_df = pd.DataFrame(results).sort_values('MAPE')
forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

print(results_df.head())
forecast_df

Average MAPE summed: 13.00%
Average MAPE daily: 43.93%
     pizza_name_id      p_d_q       P_D_Q_s  MAPE_s   MAPE
72     the_greek_l  (1, 0, 2)  (0, 0, 1, 7)   51.67  19.91
26  spinach_supr_s  (1, 0, 2)  (0, 0, 1, 7)   13.61  20.48
82  spinach_supr_l  (2, 0, 2)  (1, 0, 1, 7)   51.14  21.07
34   pep_msh_pep_l  (1, 0, 2)  (0, 0, 1, 7)   16.25  21.23
47     ckn_pesto_l  (1, 0, 2)  (0, 0, 1, 7)    7.57  23.03


,pizza_name_id,Forecast Quantity,MAPE_s,MAPE
0,bbq_ckn_l,16,13.32,51.08
1,calabrese_m,10,18.74,43.99
2,thai_ckn_m,8,7.24,33.44
3,mediterraneo_m,5,32.44,49.61
4,prsc_argla_m,12,12.76,40.79
...,...,...,...,...
86,soppressata_s,5,3.30,34.99
87,mediterraneo_s,6,2.15,34.03
88,ital_veggie_l,4,10.62,57.17
89,calabrese_s,2,10.10,76.04


#### Prophet -- 44.97%

Average MAPE summed: 12.27%  
Average MAPE: 44.97%

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('training_dataset.csv')

df["order_date"] = pd.to_datetime(df["order_date"])
df = df.sort_values("order_date")

df_daily = df.groupby(['order_date', 'pizza_name_id'])['quantity'] \
             .sum() \
             .unstack() \
             .fillna(0)
df_daily.index = pd.to_datetime(df_daily.index)

forecast = []

for pizza in df_daily.columns:
  series = df_daily[pizza]
  series = series.asfreq('D', fill_value=0)

  df_prophet = series.reset_index()
  df_prophet.columns = ['ds', 'y']

  if len(df_prophet) < 30:
        continue

  train_size = int(len(df_prophet) * 0.8)

  train_df = df_prophet.iloc[:train_size]
  test_df = df_prophet.iloc[train_size:]

  model = Prophet()
  model.fit(train_df)

  # Prediction

  future_test = model.make_future_dataframe(periods=len(test_df), freq="D")             # to generate dates for the test period
  forecast_test = model.predict(future_test)                                  # to specify the model on which dates to predict

  test_forecast = forecast_test.iloc[train_size:][["ds", "yhat"]]                    # Extract the predictions
  test_forecast['yhat'] = np.maximum(test_forecast['yhat'], 0)

  ## Evaluate

  # summed
  actual_sum  = test_df['y'].sum()
  predicted_sum  = test_forecast['yhat'].sum()

  if actual_sum  == 0:
    mape_s = np.nan
  else:
    mape_s = abs(actual_sum  - predicted_sum ) / actual_sum  * 100

  # daily
  actual_daily = test_df['y'].values
  predicted_daily = test_forecast['yhat'].values

  mask = actual_daily != 0

  if mask.sum() > 0:
    mape = mean_absolute_percentage_error(
            actual_daily[mask],
            predicted_daily[mask]
        ) * 100
  else:
    mape = np.nan

  # Retrain on FULL data                                                      # Provides the model with entire data available to forecast

  final_model = Prophet()
  final_model.fit(df_prophet)

  # Forecast Next 7 Days

  future_7 = final_model.make_future_dataframe(periods=7, freq="D")
  forecast_7 = final_model.predict(future_7)

  next_7_days = forecast_7.tail(7)[["ds", "yhat"]]
  next_7_days['yhat'] = np.maximum(next_7_days['yhat'], 0)
  forecasted_quantity = int(round(next_7_days['yhat'].sum()))

  forecast.append({
      'pizza_name_id': pizza,
      'forecasted_quantity': forecasted_quantity,
      'MAPE_s': round(mape_s, 2),
      'MAPE': round(mape, 2)
  })

forecast_df = pd.DataFrame(forecast)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE: {average_mape:.2f}%")

forecast_df

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to overr

Average MAPE summed: 12.27%
Average MAPE: 44.97%


,pizza_name_id,forecasted_quantity,MAPE_s,MAPE
0,bbq_ckn_l,16,18.08,48.42
1,bbq_ckn_m,17,8.09,56.58
2,bbq_ckn_s,9,10.71,35.50
3,big_meat_s,36,4.95,38.50
4,brie_carre_s,10,5.07,46.13
...,...,...,...,...
86,the_greek_xl,10,1.07,36.63
87,the_greek_xxl,1,0.73,91.31
88,veggie_veg_l,8,20.44,40.94
89,veggie_veg_m,12,19.65,44.68


#### LSTM -- 44.01%

Average MAPE summed: 9.79%  
Average MAPE daily: 44.01%

In [ ]:
## Import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

## Loading, Sorting, and Cleaning Data

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values('order_date')

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({'quantity': 'sum'})
      .reset_index()
)

forecast_results = list()                                         # to save the forecasts
window_size = 14

## Loop through each pizza

for pizza in daily_df['pizza_name_id'].unique():
  pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
  pizza_df = pizza_df.sort_values('order_date')
  pizza_df = pizza_df.set_index('order_date')
  pizza_df = pizza_df.asfreq('D', fill_value=0)
  pizza_df = pizza_df.reset_index()

  if len(pizza_df) < window_size + 7:
        continue

  ## Selcting the Target Variable
  sales = pizza_df[['quantity']]

  ## Train and Test Split
  train_size = int(len(sales) * 0.8)

  if train_size <= window_size:
        continue

  train_data = sales.iloc[:train_size]
  test_data  = sales.iloc[train_size:]

  ## Normalising
  scaler = MinMaxScaler()
  train_scaled = scaler.fit_transform(train_data)
  test_scaled  = scaler.transform(test_data)

  ## Creating Sequences -- Sliding Window
  full_scaled = np.vstack([train_scaled[-window_size:], test_scaled])

  X_train, y_train = [], []
  X_test, y_test = [], []

  for i in range(window_size, len(train_scaled)):
      X_train.append(train_scaled[i-window_size:i, 0])
      y_train.append(train_scaled[i, 0])

  for i in range(window_size, len(full_scaled)):
    X_test.append(full_scaled[i-window_size:i, 0])
    y_test.append(full_scaled[i, 0])

  X_train = np.array(X_train).reshape(-1, window_size, 1)
  y_train = np.array(y_train)

  X_test  = np.array(X_test).reshape(-1, window_size, 1)
  y_test  = np.array(y_test)

  ## Building the Model
  model = Sequential([
        Input(shape=(window_size, 1)),            # Input Layer
        LSTM(32),                                 # LSTM Layer
        Dropout(0.2),                             # Dropout Layer
        Dense(1)                                  # Dense Output Layer
    ])

  ## Compile the model
  model.compile(optimizer='adam', loss='mse')

  ## Early Stopping
  early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
  )

  ## Train the model
  history = model.fit(
      X_train, y_train,
      epochs=40,
      batch_size=16,
      validation_split=0.2,
      # validation_data=(X_test, y_test),
      callbacks=[early_stop],
      verbose=0
  )

  ## make predictions
  test_predictions = model.predict(X_test, verbose=0)

  ## Inverse Transform --- predictions & test
  test_predictions = scaler.inverse_transform(test_predictions)
  y_test_actual    = scaler.inverse_transform(y_test.reshape(-1,1))

  ## Evaluate

  # Summed MAPE
  actual_sum = y_test_actual.sum()
  pred_sum   = test_predictions.sum()

  if actual_sum == 0:
    mape_s = np.nan
  else:
    mape_s = abs(actual_sum - pred_sum) / actual_sum * 100

  # Daily MAPE
  mask = y_test_actual.flatten() != 0

  if mask.sum() > 0:
    mape = mean_absolute_percentage_error(
        y_test_actual[mask],
        test_predictions[mask]
    ) * 100
  else:
    mape = np.nan

  ## Forecast
  last_window = scaler.transform(sales.iloc[-window_size:])
  current_input = last_window.reshape(1, window_size, 1)
  future_preds = []

  for _ in range(7):
    next_pred = model.predict(current_input, verbose=0)[0,0]
    future_preds.append(next_pred)
    current_input = np.append(
        current_input[:,1:,:],
        [[[next_pred]]],
        axis=1
    )

  future_preds = scaler.inverse_transform(
        np.array(future_preds).reshape(-1,1))
  future_preds = np.maximum(future_preds, 0)
  forecast_sum = int(future_preds.sum().round())

  forecast_results.append({
        'pizza_name_id': pizza,
        'forecast_quantity_7_days': forecast_sum,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

forecast_df

Average MAPE summed: 9.79%
Average MAPE daily: 44.01%


,pizza_name_id,forecast_quantity_7_days,MAPE_s,MAPE
0,bbq_ckn_l,18,1.51,59.13
1,bbq_ckn_m,19,9.63,61.25
2,bbq_ckn_s,8,4.53,35.52
3,big_meat_s,34,2.00,35.24
4,brie_carre_s,9,14.23,38.61
...,...,...,...,...
86,the_greek_xl,11,0.70,44.75
87,the_greek_xxl,1,5.16,91.59
88,veggie_veg_l,8,0.04,38.22
89,veggie_veg_m,10,10.82,43.25


In [ ]:
print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

forecast_df

Average MAPE summed: 9.79%
Average MAPE daily: 44.01%


,pizza_name_id,forecast_quantity_7_days,MAPE_s,MAPE
0,bbq_ckn_l,18,1.51,59.13
1,bbq_ckn_m,19,9.63,61.25
2,bbq_ckn_s,8,4.53,35.52
3,big_meat_s,34,2.00,35.24
4,brie_carre_s,9,14.23,38.61
...,...,...,...,...
86,the_greek_xl,11,0.70,44.75
87,the_greek_xxl,1,5.16,91.59
88,veggie_veg_l,8,0.04,38.22
89,veggie_veg_m,10,10.82,43.25


### Multivariate

#### ARIMAX -- 50.92%

Average MAPE summed: 9.54%  
Average MAPE daily: 50.92%

In [ ]:
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum',
          'is_weekend': 'max',
          'is_month_start': 'max',
          'is_month_end': 'max',
          'day_of_week_num': 'max',
          'month_num': 'max',
          'week_of_year': 'max',
          'day_of_month': 'max',
          'quarter': 'max'
      })
      .reset_index()
)

daily_df = daily_df.sort_values('order_date')

results = list()
forecast_results = list()

for pizza in daily_df['pizza_name_id'].unique():

    pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
    pizza_df.set_index('order_date', inplace=True)
    pizza_df = pizza_df.asfreq('D', fill_value=0)

    y = pizza_df['quantity']
    X = pizza_df[['is_weekend', 'is_month_start', 'is_month_end',
                  'day_of_week_num', 'month_num', 'week_of_year',
                  'day_of_month', 'quarter']]

    if len(y) < 60:
        continue

    ## d value

    d = 1 if adfuller(y)[1] > 0.05 else 0

    ## Train Test

    train_size = int(len(y) * 0.8)

    y_train = y.iloc[:train_size]
    y_test  = y.iloc[train_size:]

    X_train = X.iloc[:train_size]
    X_test  = X.iloc[train_size:]

    ## p, q values -- AIC

    p = q = range(0, 3)

    best_aic = np.inf
    best_order = None

    for i, j in itertools.product(p, q):
      try:
          model = SARIMAX(
              y_train,
              exog=X_train,
              order=(i, d, j),
              enforce_stationarity=False,
              enforce_invertibility=False)

          model_fit = model.fit(disp=False)

          if model_fit.aic < best_aic:
              best_aic = model_fit.aic
              best_order = (i, d, j)

      except:
          continue

    ## Train

    final_model = SARIMAX(
    y_train,
    exog=X_train,
    order=best_order,
    enforce_stationarity=False,
    enforce_invertibility=False)

    final_fit = final_model.fit(disp=False)

    # Test Prediction
    test_pred = final_fit.predict(
      start=len(y_train),
      end=len(y_train) + len(y_test) - 1,
      exog=X_test
    )

    ## Evaluation

    # Summed MAPE

    actual_sum = y_test.sum()
    predicted_sum = test_pred.sum()

    if actual_sum == 0:
        mape_s = np.nan
    else:
        mape_s = abs((actual_sum - predicted_sum) / actual_sum) * 100

    # Daily MAPE

    mask = y_test.values != 0
    if mask.sum() > 0:
        mape = mean_absolute_percentage_error(
            y_test.values[mask],
            test_pred.values[mask]
        ) * 100
    else:
        mape = np.nan

    ## Forecast

    future_dates = pd.date_range(
      start=y.index[-1] + pd.Timedelta(days=1),
      periods=7,
      freq='D'
    )

    future_X = pd.DataFrame({
      'is_weekend': (future_dates.weekday >= 5).astype(int),
      'is_month_start': future_dates.is_month_start.astype(int),
      'is_month_end': future_dates.is_month_end.astype(int),
        'day_of_week_num': future_dates.weekday,
        'month_num': future_dates.month,
        'week_of_year': X['week_of_year'].iloc[0],
        'day_of_month': future_dates.day,
        'quarter': future_dates.quarter
  }, index=future_dates)

    future_forecast = final_fit.forecast(steps=7, exog=future_X)
    future_forecast_daily = future_forecast.tail(7)
    future_forecast = future_forecast_daily.sum().round().astype(int)

    results.append({
      'pizza_name_id': pizza,
      'p_d_q': best_order,
      'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
  })

    forecast_results.append({
        'pizza_name_id': pizza,
        'Forecast Quantity': future_forecast,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

results_df = pd.DataFrame(results).sort_values('MAPE')
forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

print(results_df.head())
forecast_df

Average MAPE summed: 9.54%
Average MAPE daily: 50.92%
     pizza_name_id      p_d_q  MAPE_s   MAPE
90   the_greek_xxl  (0, 0, 1)    4.48   4.48
48   ckn_alfredo_s  (0, 0, 0)   14.65  20.49
37  green_garden_l  (0, 0, 0)   22.92  21.88
70   ital_cpcllo_s  (0, 0, 0)   27.51  30.50
54      mexicana_s  (2, 0, 0)    0.35  31.10


,pizza_name_id,Forecast Quantity,MAPE_s,MAPE
0,bbq_ckn_l,21,8.80,60.59
1,calabrese_m,14,10.55,52.53
2,thai_ckn_m,12,2.99,52.38
3,mediterraneo_m,9,2.86,48.47
4,prsc_argla_m,14,3.12,46.17
...,...,...,...,...
86,soppressata_s,9,14.31,48.37
87,mediterraneo_s,9,6.49,43.07
88,ital_veggie_l,8,9.45,36.30
89,calabrese_s,79,15.29,34.50


#### SARIMAX -- 51.18%

Average MAPE summed: 7.64%  
Average MAPE daily: 51.18%

In [ ]:
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum',
          'is_weekend': 'max',
          'is_month_start': 'max',
          'is_month_end': 'max',
          'day_of_week_num': 'max',
          'month_num': 'max',
          'week_of_year': 'max',
          'day_of_month': 'max',
          'quarter': 'max'
      })
      .reset_index()
)

daily_df = daily_df.sort_values('order_date')

results = list()
forecast_results = list()

for pizza in daily_df['pizza_name_id'].unique():

    pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
    pizza_df.set_index('order_date', inplace=True)
    pizza_df = pizza_df.asfreq('D', fill_value=0)

    y = pizza_df['quantity']
    X = pizza_df[['is_weekend', 'is_month_start', 'is_month_end',
                  'day_of_week_num', 'month_num', 'week_of_year',
                  'day_of_month', 'quarter']]

    if len(y) < 60:
        continue


    ## d value

    d = 1 if adfuller(y)[1] > 0.05 else 0

    ## D value

    D = 1 if adfuller(y.diff(7).dropna())[1] > 0.05 else 0

    s = 7

    ## Train Test

    train_size = int(len(y) * 0.8)

    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]

    ## p, q, P, Q values -- AIC

    p = q = range(0, 3)
    P = Q = range(0, 2)

    best_aic = np.inf
    best_order = None
    best_seasonal = None

    for i, j, m, n in itertools.product(p, q, P, Q):
      try:
          model = SARIMAX(
              y_train,
              exog=X_train,
              order=(i, d, j),
              seasonal_order=(m, D, n, s),
              enforce_stationarity=False,
              enforce_invertibility=False
          )

          model_fit = model.fit(disp=False)

          if model_fit.aic < best_aic:
              best_aic = model_fit.aic
              best_order = (i, d, j)
              best_seasonal = (m, D, n, s)

      except:
          continue

    ## Train

    final_model = SARIMAX(
    y_train,
    exog=X_train,
    order=best_order,
    seasonal_order=best_seasonal,
    enforce_stationarity=False,
    enforce_invertibility=False
    )

    final_fit = final_model.fit(disp=False)

    # Test Prediction
    test_pred = final_fit.predict(
      start=len(y_train),
      end=len(y_train) + len(y_test) - 1,
      exog=X_test
    )

    ## Evaluation

    # Summed MAPE
    actual_sum = y_test.sum()
    predicted_sum = test_pred.sum()

    if actual_sum == 0:
        mape_s = np.nan
    else:
        mape_s = abs((actual_sum - predicted_sum) / actual_sum) * 100

    # Daily MAPE
    mask = y_test.values != 0

    if mask.sum() > 0:
        mape = mean_absolute_percentage_error(
            y_test.values[mask],
            test_pred.values[mask]
        ) * 100
    else:
        mape = np.nan

    ## Forecast

    future_dates = pd.date_range(
      start=y.index[-1] + pd.Timedelta(days=1),
      periods=7,
      freq='D'
    )

    future_X = pd.DataFrame({
      'is_weekend': (future_dates.weekday >= 5).astype(int),
      'is_month_start': future_dates.is_month_start.astype(int),
      'is_month_end': future_dates.is_month_end.astype(int),
        'day_of_week_num': future_dates.weekday,
        'month_num': future_dates.month,
        'week_of_year': X['week_of_year'].iloc[0],
        'day_of_month': future_dates.day,
        'quarter': future_dates.quarter
  }, index=future_dates)

    future_forecast = final_fit.forecast(steps=7, exog=future_X)
    future_forecast_daily = future_forecast.tail(7)
    future_forecast = future_forecast_daily.sum().round().astype(int)


    results.append({
      'pizza_name_id': pizza,
      'p_d_q': best_order,
      'P_D_Q_s': best_seasonal,
      'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
  })

    forecast_results.append({
        'pizza_name_id': pizza,
        'Forecast Quantity': future_forecast,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

results_df = pd.DataFrame(results).sort_values('MAPE')
forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

print(results_df.head())
forecast_df

Average MAPE summed: 7.64%
Average MAPE daily: 51.18%
     pizza_name_id      p_d_q       P_D_Q_s  MAPE_s   MAPE
16   ckn_alfredo_s  (0, 0, 0)  (0, 0, 0, 7)   14.65  20.49
6   green_garden_l  (0, 0, 0)  (0, 0, 0, 7)   22.92  21.88
27     calabrese_s  (0, 0, 0)  (0, 0, 0, 7)   15.29  34.50
26     calabrese_l  (0, 0, 1)  (1, 0, 1, 7)    7.85  35.46
23   ckn_alfredo_l  (0, 0, 0)  (0, 0, 0, 7)   20.96  37.53


,pizza_name_id,Forecast Quantity,MAPE_s,MAPE
0,bbq_ckn_l,21,7.87,61.22
1,green_garden_m,10,1.42,45.66
2,big_meat_s,37,3.15,40.43
3,bbq_ckn_s,14,6.67,43.13
4,cali_ckn_s,13,0.77,50.03
5,hawaiian_m,120,9.84,47.09
6,green_garden_l,90,22.92,21.88
7,ckn_pesto_m,10,1.97,46.33
8,four_cheese_m,13,10.58,58.99
9,classic_dlx_l,12,15.32,41.22


#### Prophet -- 51.33%

Average MAPE summed: 10.16%  
Average MAPE: 51.33%

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')

df["order_date"] = pd.to_datetime(df["order_date"])
df = df.sort_values("order_date")

pizza_list = df['pizza_name_id'].unique()

forecast = list()

regressors = ['day_of_week_num', 'is_weekend', 'month_num',
       'week_of_year', 'day_of_month', 'quarter', 'is_month_start',
        'is_month_end']

col_to_group = ['order_date', 'day_of_week_num', 'is_weekend', 'month_num',
       'week_of_year', 'day_of_month', 'quarter', 'is_month_start',
       'is_month_end']

for pizza in pizza_list:
  pizza_df = df[df["pizza_name_id"] == pizza]
  pizza_df = (pizza_df.groupby(col_to_group, as_index=False).agg({'quantity': 'sum'}))
  pizza_df = pizza_df.sort_values("order_date")

  if len(pizza_df) < 30:
        continue

  # Train
  pizza_df = pizza_df.rename(columns={
      "order_date": "ds",
      "quantity": "y"})

  train_size = int(len(pizza_df) * 0.8)

  train_df = pizza_df.iloc[:train_size]
  test_df = pizza_df.iloc[train_size:]

  model = Prophet()

  for col in regressors:
        model.add_regressor(col)

  model.fit(train_df[['ds','y'] + regressors])

  # Prediction
  future_test = test_df[['ds'] + regressors]
  forecast_test = model.predict(future_test)                                  # to specify the model on which dates to predict
  forecast_test['yhat'] = np.maximum(forecast_test['yhat'], 0)

  ## Evaluate
  # Sum
  actual_sum = test_df['y'].sum()
  predicted_sum = forecast_test['yhat'].sum()

  if actual_sum == 0:
      mape_s = np.nan
  else:
      mape_s = abs(actual_sum - predicted_sum) / actual_sum * 100

  # Individual

  actual_daily = test_df['y'].values
  predicted_daily = forecast_test['yhat'].values

  mask = actual_daily != 0

  if mask.sum() > 0:
      mape = mean_absolute_percentage_error(
          actual_daily[mask],
          predicted_daily[mask]
      ) * 100
  else:
      mape = np.nan

  # Retrain on FULL data                                                      # Provides the model with entire data available to forecast

  final_model = Prophet()

  for col in regressors:
        final_model.add_regressor(col)

  final_model.fit(pizza_df[['ds', 'y'] + regressors])

  future_dates = pd.date_range(
        start=pizza_df['ds'].max() + pd.Timedelta(days=1),
        periods=7,
        freq='D'
    )

  future_7_df = pd.DataFrame({'ds': future_dates})
  future_7_df['day_of_week_num'] = future_dates.weekday
  future_7_df['is_weekend'] = (future_dates.weekday >= 5).astype(int)
  future_7_df['month_num'] = future_dates.month
  future_7_df['week_of_year'] = (future_dates.isocalendar().week.to_numpy(dtype=int))
  future_7_df['day_of_month'] = future_dates.day
  future_7_df['quarter'] = future_dates.quarter
  future_7_df['is_month_start'] = future_dates.is_month_start.astype(int)
  future_7_df['is_month_end'] = future_dates.is_month_end.astype(int)

  forecast_7 = final_model.predict(future_7_df)
  forecast_7['yhat'] = np.maximum(forecast_7['yhat'], 0)
  forecasted_quantity = int(round(forecast_7['yhat'].sum()))

  forecast.append({
      'pizza_name_id': pizza,
      'forecasted_quantity': forecasted_quantity,
      'MAPE_s': round(mape_s, 2),
      'MAPE': round(mape, 2)
  })

forecast_df = pd.DataFrame(forecast)
average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()
print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE: {average_mape:.2f}%")
forecast_df

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to overr

Average MAPE summed: 10.16%
Average MAPE: 51.33%


,pizza_name_id,forecasted_quantity,MAPE_s,MAPE
0,bbq_ckn_l,12,11.52,56.72
1,mexicana_l,6,15.69,67.28
2,southw_ckn_m,18,16.03,69.72
3,mediterraneo_m,13,10.80,52.98
4,calabrese_m,13,11.17,51.39
...,...,...,...,...
85,veggie_veg_m,20,24.39,44.99
86,mediterraneo_s,16,0.83,43.60
87,soppressata_s,4,12.11,46.16
88,ital_veggie_l,10,9.41,36.07


#### LSTM -- 41.20%

Average MAPE summed: 14.03%  
Average MAPE daily: 41.20%

In [ ]:
## Import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

## Loading, Sorting, and Cleaning Data

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values('order_date')

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum',
          'is_weekend': 'max',
          'is_month_start': 'max',
          'is_month_end': 'max',
          'day_of_week_num': 'max',
          'month_num': 'max',
          'week_of_year': 'max',
          'day_of_month': 'max',
          'quarter': 'max'
      })
      .reset_index())

daily_df = daily_df.sort_values('order_date')
daily_df = daily_df.reset_index(drop=True)

forecast_results = list()                                         # to save the forecasts

## Loop through each pizza

for pizza in daily_df['pizza_name_id'].unique():
  pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
  pizza_df = pizza_df.sort_values('order_date')

  if len(pizza_df) < 60:
        continue

  pizza_df['lag_7'] = pizza_df['quantity'].shift(7)
  pizza_df['rolling_mean_7'] = pizza_df['quantity'].rolling(7).mean()
  pizza_df = pizza_df.dropna().reset_index(drop=True)

  features = ['is_weekend', 'is_month_start', 'is_month_end',
              'day_of_week_num', 'month_num', 'week_of_year', 'day_of_month',
              'quarter', 'lag_7', 'rolling_mean_7']
  target = 'quantity'

  X = pizza_df[features]
  y = pizza_df[target]

  train_size = int(len(X) * 0.8)

  X_train_raw = X.iloc[:train_size]
  X_test_raw  = X.iloc[train_size:]
  y_train_raw = y.iloc[:train_size]
  y_test_raw  = y.iloc[train_size:]

  scaler_X = MinMaxScaler()
  scaler_y = MinMaxScaler()

  X_train_scaled = scaler_X.fit_transform(X_train_raw)
  X_test_scaled  = scaler_X.transform(X_test_raw)

  y_train_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1))
  y_test_scaled  = scaler_y.transform(y_test_raw.values.reshape(-1, 1))


  def create_sequences(X, y, window):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i-window:i])   # ALL features
        ys.append(y[i])
    return np.array(Xs), np.array(ys)

  window_size = 14

  X_train, y_train = create_sequences(X_train_scaled, y_train_scaled, window_size)
  X_test, y_test   = create_sequences(
        np.vstack([X_train_scaled[-window_size:], X_test_scaled]),
        np.vstack([y_train_scaled[-window_size:], y_test_scaled]),
        window_size
    )

  model = Sequential([
    Input(shape=(window_size, X_train.shape[2])),  # n_features
    LSTM(32),
    Dropout(0.2),
    Dense(1)
])

  model.compile(optimizer='adam', loss='mae')

  ## Early Stopping
  early_stop = EarlyStopping(
  monitor='val_loss',
  patience=5,
  restore_best_weights=True
  )


  model.fit(
    X_train, y_train,
    epochs=40,
    batch_size=16,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=0
)

  preds = scaler_y.inverse_transform(model.predict(X_test))
  actual = scaler_y.inverse_transform(y_test)

  # Daily MAPE
  mape = mean_absolute_percentage_error(actual, preds) * 100

  # Summed MAPE
  actual_sum = actual.sum()
  pred_sum = preds.sum()
  mape_s = abs(actual_sum - pred_sum) / actual_sum * 100

  last_window = np.vstack([X_train_scaled[-window_size:], X_test_scaled])[-window_size:]
  last_date = pizza_df["order_date"].iloc[-1]
  last_qty = pizza_df['quantity'].iloc[-window_size:].values

  future_preds = []

  for i in range(7):
      next_scaled = model.predict(last_window.reshape(1, window_size, -1), verbose = 0)[0, 0]
      next_qty = scaler_y.inverse_transform([[next_scaled]])[0, 0]
      future_preds.append(next_qty)

      # Update window
      next_date = last_date + pd.Timedelta(days=1)


      lag_7 = last_qty[-7]
      rolling_mean_7 = last_qty[-7:].mean()

      new_row = pd.DataFrame([[
            int(next_date.weekday() >= 5),
            int(next_date.is_month_start),
            int(next_date.is_month_end),
            next_date.weekday(),
            next_date.month,
            int(next_date.isocalendar().week),
            next_date.day,
            next_date.quarter,
            lag_7,
            rolling_mean_7
        ]], columns=features)

      new_row_scaled = scaler_X.transform(new_row)
      last_window = np.vstack([last_window[1:], new_row_scaled])
      last_qty = np.append(last_qty[1:], next_qty)
      last_date = next_date

  forecast_sum = int(round(sum(future_preds)))

  forecast_results.append({
        "pizza_name_id": pizza,
        "forecast_quantity_7_days": forecast_sum,
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

forecast_df = pd.DataFrame(forecast_results)

average_mape_s = forecast_df['MAPE_s'].mean()
average_mape = forecast_df['MAPE'].mean()

print(f"Average MAPE summed: {average_mape_s:.2f}%")
print(f"Average MAPE daily: {average_mape:.2f}%")

forecast_df

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 268ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 258ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 207ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 255ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 192ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 252ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 299ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s

,pizza_name_id,forecast_quantity_7_days,MAPE_s,MAPE
0,bbq_ckn_l,17,2.57,62.14
1,calabrese_m,9,1.56,58.71
2,thai_ckn_m,7,20.11,35.11
3,mediterraneo_m,11,28.94,26.92
4,prsc_argla_m,9,9.42,43.73
...,...,...,...,...
85,veggie_veg_m,14,16.40,44.42
86,soppressata_s,10,1.20,35.05
87,mediterraneo_s,6,30.36,29.32
88,ital_veggie_l,8,14.82,23.26


### Random Forest Regressor -- 51.18%

Average MAPE summed: 8.26%  
Average MAPE daily: 51.18%

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')
df['order_date'] = pd.to_datetime(df['order_date'])

# Aggregate daily per pizza
daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum',
          'is_weekend': 'max',
          'is_month_start': 'max',
          'is_month_end': 'max',
          'day_of_week_num': 'max',
          'month_num': 'max',
          'week_of_year': 'max',
          'day_of_month': 'max',
          'quarter': 'max'
      })
      .reset_index()
)

daily_df = daily_df.sort_values('order_date')
forecast_results = list()

for pizza in daily_df['pizza_name_id'].unique():
    pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
    pizza_df = pizza_df.set_index('order_date').asfreq('D', fill_value=0).reset_index()

    if len(pizza_df) < 60:
        continue

    # Lag & rolling features
    pizza_df['lag_7'] = pizza_df['quantity'].shift(7)
    pizza_df['lag_14'] = pizza_df['quantity'].shift(14)
    pizza_df['rolling_mean_7'] = pizza_df['quantity'].rolling(7).mean()
    pizza_df['rolling_mean_14'] = pizza_df['quantity'].rolling(14).mean()
    pizza_df = pizza_df.dropna().reset_index(drop=True)

    features = ['is_weekend', 'is_month_start', 'is_month_end',
                'day_of_week_num', 'month_num', 'week_of_year',
                'day_of_month', 'quarter', 'lag_7', 'lag_14',
                'rolling_mean_7', 'rolling_mean_14']

    X = pizza_df[features]
    y = pizza_df['quantity']

    # Train/Test split
    train_size = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    # Random Forest Model
    model = RandomForestRegressor(
        n_estimators=300,
        max_depth=10,
        min_samples_leaf=5,
        max_features=0.8,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Evaluation
    preds = model.predict(X_test)
    preds = np.maximum(preds, 0)
    mask = y_test.values != 0
    mape = mean_absolute_percentage_error(y_test.values[mask], preds[mask]) * 100

    actual_sum = y_test.sum()
    pred_sum = preds.sum()
    mape_s = abs(actual_sum - pred_sum) / actual_sum * 100 if actual_sum > 0 else np.nan

    # 7-Day Forecast
    last_row = pizza_df.iloc[-1:].copy()
    future_preds = []

    for _ in range(7):
        X_last = last_row[features]
        next_pred = model.predict(X_last)[0]
        next_pred = max(0, next_pred)
        future_preds.append(next_pred)

        next_date = last_row['order_date'].values[0] + np.timedelta64(1, 'D')
        new_row = last_row.copy()
        new_row['order_date'] = next_date
        new_row['quantity'] = next_pred

        # Update lags
        new_row['lag_14'] = new_row['lag_7']
        new_row['lag_7'] = next_pred

        # Update rolling means
        last_7 = pizza_df['quantity'].iloc[-6:].tolist() + [next_pred]
        last_14 = pizza_df['quantity'].iloc[-13:].tolist() + [next_pred]

        new_row['rolling_mean_7'] = np.mean(last_7)
        new_row['rolling_mean_14'] = np.mean(last_14)

        pizza_df = pd.concat([pizza_df, new_row], ignore_index=True)
        last_row = new_row

    forecast_results.append({
        'pizza_name_id': pizza,
        'Forecast Quantity': int(round(sum(future_preds))),
        'MAPE_s': round(mape_s, 2),
        'MAPE': round(mape, 2)
    })

forecast_df = pd.DataFrame(forecast_results)

print(f"Average MAPE summed: {forecast_df['MAPE_s'].mean():.2f}%")
print(f"Average MAPE daily: {forecast_df['MAPE'].mean():.2f}%")
forecast_df

Average MAPE summed: 8.26%
Average MAPE daily: 51.18%


,pizza_name_id,Forecast Quantity,MAPE_s,MAPE
0,bbq_ckn_l,16,2.30,61.44
1,calabrese_m,15,19.24,74.87
2,thai_ckn_m,13,10.83,55.26
3,mediterraneo_m,13,8.43,48.25
4,prsc_argla_m,20,6.78,49.53
...,...,...,...,...
86,soppressata_s,12,29.67,55.27
87,mediterraneo_s,11,2.34,38.74
88,ital_veggie_l,12,21.93,46.36
89,calabrese_s,9,7.67,21.48


### XGBoost -- 59.63%

Average MAPE summed: 15.57%  
Average MAPE daily: 59.63%

In [ ]:
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error

df = pd.read_csv('train_data_without_scaling.csv')

df['order_date'] = pd.to_datetime(df['order_date'])

daily_df = (
    df.groupby(['pizza_name_id', 'order_date'])
      .agg({
          'quantity': 'sum',
          'is_weekend': 'max',
          'is_month_start': 'max',
          'is_month_end': 'max',
          'day_of_week_num': 'max',
          'month_num': 'max',
          'week_of_year': 'max',
          'day_of_month': 'max',
          'quarter': 'max'
      })
      .reset_index()
)

daily_df = daily_df.sort_values('order_date')

forecast_results = list()

for pizza in daily_df['pizza_name_id'].unique():

    pizza_df = daily_df[daily_df['pizza_name_id'] == pizza].copy()
    pizza_df = pizza_df.set_index('order_date').asfreq('D', fill_value=0).reset_index()

    if len(pizza_df) < 60:
        continue

    # Lag & rolling features
    pizza_df['lag_7'] = pizza_df['quantity'].shift(7)
    pizza_df['lag_14'] = pizza_df['quantity'].shift(14)
    pizza_df['rolling_mean_7'] = pizza_df['quantity'].rolling(7).mean()
    pizza_df['rolling_mean_14'] = pizza_df['quantity'].rolling(14).mean()

    pizza_df = pizza_df.dropna().reset_index(drop=True)

    features = ['is_weekend', 'is_month_start', 'is_month_end',
                'day_of_week_num', 'month_num', 'week_of_year',
                'day_of_month', 'quarter', 'lag_7', 'lag_14',
                'rolling_mean_7', 'rolling_mean_14']

    X = pizza_df[features]
    y = pizza_df['quantity']

    # Train Test Split

    train_size = int(len(X) * 0.8)

    X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
    y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

    # Model Training
    model = XGBRegressor(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    )

    model.fit(X_train, y_train)

    # Evaluation
    preds = model.predict(X_test)
    preds = np.maximum(preds, 0)

    # Daily MAPE
    mask = y_test.values != 0
    mape = mean_absolute_percentage_error(y_test.values[mask],preds[mask]) * 100

    # Summed MAPE
    actual_sum = y_test.sum()
    pred_sum = preds.sum()

    mape_s = abs(actual_sum - pred_sum) / actual_sum * 100 if actual_sum > 0 else np.nan

    # Forecast
    last_row = pizza_df.iloc[-1:].copy()
    future_preds = []

    for _ in range(7):

        X_last = last_row[features]
        next_pred = model.predict(X_last)[0]
        next_pred = max(0, next_pred)
        future_preds.append(next_pred)

        next_date = last_row['order_date'].values[0] + np.timedelta64(1, 'D')

        new_row = last_row.copy()
        new_row['order_date'] = next_date
        new_row['quantity'] = next_pred

        # Update lags
        new_row['lag_14'] = new_row['lag_7']
        new_row['lag_7'] = next_pred

        # Update rolling features (SCALAR!)
        last_7 = pizza_df['quantity'].iloc[-6:].tolist() + [next_pred]
        last_14 = pizza_df['quantity'].iloc[-13:].tolist() + [next_pred]

        new_row['rolling_mean_7'] = np.mean(last_7)
        new_row['rolling_mean_14'] = np.mean(last_14)

        pizza_df = pd.concat([pizza_df, new_row], ignore_index=True)
        last_row = new_row


    forecast_results.append({
    'pizza_name_id': pizza,
    'Forecast Quantity': int(round(sum(future_preds))),
    'MAPE_s': round(mape_s, 2),
    'MAPE': round(mape, 2)
    })

forecast_df = pd.DataFrame(forecast_results)

print(f"Average MAPE summed: {forecast_df['MAPE_s'].mean():.2f}%")
print(f"Average MAPE daily: {forecast_df['MAPE'].mean():.2f}%")

forecast_df

Average MAPE summed: 15.57%
Average MAPE daily: 59.63%


,pizza_name_id,Forecast Quantity,MAPE_s,MAPE
0,bbq_ckn_l,17,12.40,72.33
1,calabrese_m,11,14.16,72.81
2,thai_ckn_m,15,27.84,70.50
3,mediterraneo_m,14,6.61,47.62
4,prsc_argla_m,14,0.65,59.69
...,...,...,...,...
86,soppressata_s,13,31.54,56.93
87,mediterraneo_s,8,4.64,34.62
88,ital_veggie_l,16,36.93,57.95
89,calabrese_s,7,0.56,21.71


### Single Model

#### Random Forest

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("train_data_without_scaling.csv")
df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values(['pizza_name_id', 'order_date']).reset_index(drop=True)

# ENCODE pizza_name_id

le_pizza = LabelEncoder()
df['pizza_name_id_enc'] = le_pizza.fit_transform(df['pizza_name_id'])


# Create Lag & Rolling Features

df['lag_1'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1)
df['lag_7'] = df.groupby('pizza_name_id_enc')['quantity'].shift(7)
df['lag_14'] = df.groupby('pizza_name_id_enc')['quantity'].shift(14)
df['lag_28'] = df.groupby('pizza_name_id_enc')['quantity'].shift(28)

df['rolling_7_std'] = (
    df.groupby('pizza_name_id_enc')['quantity']
      .shift(1)
      .rolling(7)
      .std()
)

df['rolling_14_mean'] = (
    df.groupby('pizza_name_id_enc')['quantity']
      .shift(1)
      .rolling(14)
      .mean()
)

df['rolling_28_mean'] = (
    df.groupby('pizza_name_id_enc')['quantity']
      .shift(1)
      .rolling(28)
      .mean()
)

df['rolling_28_std'] = (
    df.groupby('pizza_name_id_enc')['quantity']
      .shift(1)
      .rolling(28)
      .std()
)

df['qty_vs_14d_avg'] = df['quantity'] / (df['rolling_14_mean'] + 1e-5)

# Lag of Same Weekday
df['lag_prev_same_weekday'] = df.groupby(
    ['pizza_name_id_enc', 'day_of_week_num']
)['quantity'].shift(1)

df = df.dropna().reset_index(drop=True)             # Drop NA Rows (from lags)

pizza_mean = (
    df.groupby('pizza_name_id')['quantity']
      .mean()
      .rename('pizza_mean_qty')
)

df = df.merge(pizza_mean, on='pizza_name_id', how='left')


# Train–Test Split

split_date = df['order_date'].quantile(0.80)

train_df = df[df['order_date'] <= split_date]
test_df = df[df['order_date'] > split_date].copy()

# Feature & Target Selection

target = 'quantity'

drop_cols = [
    'quantity',
    'order_date',
    'pizza_name_id']

features = [col for col in df.columns if col not in drop_cols]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Train Random Forest

rf_model = RandomForestRegressor(
    n_estimators=800,        # more trees → smoother forecast
    max_depth=18,            # avoid overfitting
    min_samples_leaf=5,      # smoother demand curves
    max_features=0.7,        # reduce noise
    random_state=42,
    n_jobs=-1)

weights = np.where(
    train_df['order_date'] > split_date - pd.Timedelta(days=60),
    1.5,
    1.0
)

rf_model.fit(X_train, y_train, sample_weight=weights)

# Test - Prediction

test_df['prediction'] = rf_model.predict(X_test)
test_df['prediction'] = test_df['prediction'].clip(lower=0)

# Daily MAPE on Test Data

daily_mape = (
    test_df
    .groupby('order_date')[['quantity', 'prediction']]
    .apply(lambda x: mean_absolute_percentage_error(
        x['quantity'], x['prediction']
    ) * 100)
    .reset_index()
)

daily_mape.columns = ['date', 'daily_mape_pct']
daily_mape['daily_mape_pct'] = daily_mape['daily_mape_pct'].round(2)

# TEST-PERIOD SUM MAPE

actual_sum = test_df.groupby('order_date')['quantity'].sum()
pred_sum   = test_df.groupby('order_date')['prediction'].sum()

mape_s = mean_absolute_percentage_error(actual_sum, pred_sum) * 100
mape_s = round(mape_s, 2)

wmape = (
    np.abs(test_df['quantity'] - test_df['prediction']).sum() /
    test_df['quantity'].sum()
) * 100

print("WMAPE (%):", round(wmape, 2))

print("TEST PERIOD SUM MAPE (%):", mape_s)
avg_daily_mape = daily_mape['daily_mape_pct'].mean()
avg_daily_mape = round(avg_daily_mape, 2)

print("AVERAGE DAILY MAPE (%):", avg_daily_mape)

print("\nDaily MAPE (%):")
print(daily_mape.head(10))

print("\nDaily MAPE Summary (%):")
print(daily_mape['daily_mape_pct'].describe().round(2))

# 7-DAY FORECAST FOR EACH PIZZA

last_date = df['order_date'].max()
pizza_ids = df[['pizza_name_id', 'pizza_name_id_enc']].drop_duplicates()
forecast_rows = []

for _, row_id in pizza_ids.iterrows():

    pid = row_id['pizza_name_id']
    pid_enc = row_id['pizza_name_id_enc']

    hist = df[df['pizza_name_id_enc'] == pid_enc].copy()

    for i in range(1, 8):

        next_date = last_date + pd.Timedelta(days=i)
        row = hist.iloc[-1:].copy()
        row['order_date'] = next_date

        # Calendar features
        row['day_of_week_num'] = next_date.weekday()
        row['is_weekend'] = int(next_date.weekday() >= 5)
        row['month_num'] = next_date.month
        row['week_of_year'] = int(next_date.isocalendar().week)
        row['day_of_month'] = next_date.day
        row['quarter'] = next_date.quarter
        row['is_month_start'] = int(next_date.is_month_start)
        row['is_month_end'] = int(next_date.is_month_end)

        # Lag features
        row['lag_1']  = hist.iloc[-1]['quantity']
        row['lag_7']  = hist['quantity'].iloc[-7] if len(hist) >= 7 else hist.iloc[0]['quantity']
        row['lag_14'] = hist['quantity'].iloc[-14] if len(hist) >= 14 else hist.iloc[0]['quantity']
        row['lag_28'] = hist['quantity'].iloc[-28] if len(hist) >= 28 else hist.iloc[0]['quantity']

        row['rolling_7_std'] = hist['quantity'].tail(7).std()
        row['rolling_14_mean'] = hist['quantity'].tail(14).mean()
        row['rolling_28_mean'] = hist['quantity'].tail(28).mean()
        row['rolling_28_std']  = hist.tail(28)['quantity'].std()

        row['qty_vs_14d_avg'] = row['lag_1'] / (row['rolling_14_mean'] + 1e-5)

        same_day_hist = hist[hist['day_of_week_num'] == next_date.weekday()]
        row['lag_prev_same_weekday'] = same_day_hist.iloc[-1]['quantity'] if len(same_day_hist) > 0 else hist.iloc[-7]['quantity']

        row['pizza_mean_qty'] = pizza_mean.loc[pid]

        X_next = row[features]
        pred_qty = rf_model.predict(X_next)[0]
        pred_qty = max(0, pred_qty)

        row['quantity'] = pred_qty
        hist = pd.concat([hist, row], ignore_index=True)

        forecast_rows.append([pid, next_date, pred_qty])

forecast_df = pd.DataFrame(
    forecast_rows,
    columns=['pizza_name_id', 'forecast_date', 'predicted_quantity']
)

final_forecast = (
    forecast_df
    .groupby('pizza_name_id')['predicted_quantity']
    .sum()
    .reset_index()
)

final_forecast['predicted_quantity'] = final_forecast['predicted_quantity'].round().astype(int)

final_forecast.head()

WMAPE (%): 0.03
TEST PERIOD SUM MAPE (%): 0.03
AVERAGE DAILY MAPE (%): 0.02

Daily MAPE (%):
        date  daily_mape_pct
0 2015-10-22            0.00
1 2015-10-23            0.00
2 2015-10-24            0.02
3 2015-10-25            0.03
4 2015-10-27            0.01
5 2015-10-28            0.01
6 2015-10-29            0.03
7 2015-10-30            0.01
8 2015-10-31            0.05
9 2015-11-01            0.03

Daily MAPE Summary (%):
count    68.00
mean      0.02
std       0.02
min       0.00
25%       0.01
50%       0.01
75%       0.02
max       0.11
Name: daily_mape_pct, dtype: float64


,pizza_name_id,predicted_quantity
0,bbq_ckn_l,14
1,bbq_ckn_m,7
2,bbq_ckn_s,7
3,big_meat_s,7
4,brie_carre_s,7


In [ ]:
feat_imp = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

feat_imp

,feature,importance
8,day_of_month,0.166420
6,week_of_year,0.137717
14,pizza_name_id_enc,0.136648
7,order_hour,0.097995
3,day_of_week_num,0.094238
0,unit_price,0.073065
5,month_num,0.072433
2,pizza_category,0.035787
20,rolling_14_mean,0.034170
9,quarter,0.030717


In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("train_data_without_scaling.csv")
df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values(['pizza_name_id', 'order_date']).reset_index(drop=True)

# ENCODE pizza_name_id

le_pizza = LabelEncoder()
df['pizza_name_id_enc'] = le_pizza.fit_transform(df['pizza_name_id'])

# Create Lag & Rolling Features

df['lag_1'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1)
df['lag_7'] = df.groupby('pizza_name_id_enc')['quantity'].shift(7)
df['rolling_7_mean'] = (
    df.groupby('pizza_name_id_enc')['quantity']
      .shift(1)
      .rolling(7)
      .mean())
df = df.dropna().reset_index(drop=True)             # Drop NA Rows (from lags)

# Train–Test Split

split_date = df['order_date'].quantile(0.80)

train_df = df[df['order_date'] <= split_date]
test_df = df[df['order_date'] > split_date].copy()

# Feature & Target Selection

target = 'quantity'

drop_cols = [
    'quantity',
    'order_date',
    'pizza_name_id']

features = [col for col in df.columns if col not in drop_cols]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

# Train Random Forest

rf_model = RandomForestRegressor(
    n_estimators=400,
    max_depth=15,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1)
rf_model.fit(X_train, y_train)

# Test - Prediction

test_df['prediction'] = rf_model.predict(X_test)
test_df['prediction'] = test_df['prediction'].clip(lower=0)

# Daily MAPE on Test Data

daily_mape = (
    test_df
    .groupby('order_date')[['quantity', 'prediction']]
    .apply(lambda x: mean_absolute_percentage_error(
        x['quantity'], x['prediction']
    ) * 100)
    .reset_index()
)

daily_mape.columns = ['date', 'daily_mape_pct']
daily_mape['daily_mape_pct'] = daily_mape['daily_mape_pct'].round(2)

# TEST-PERIOD SUM MAPE

actual_sum = test_df.groupby('order_date')['quantity'].sum()
pred_sum   = test_df.groupby('order_date')['prediction'].sum()

mape_s = mean_absolute_percentage_error(actual_sum, pred_sum) * 100
mape_s = round(mape_s, 2)

print("TEST PERIOD SUM MAPE (%):", mape_s)
avg_daily_mape = daily_mape['daily_mape_pct'].mean()
avg_daily_mape = round(avg_daily_mape, 2)

print("AVERAGE DAILY MAPE (%):", avg_daily_mape)

print("\nDaily MAPE (%):")
print(daily_mape.head(10))

print("\nDaily MAPE Summary (%):")
print(daily_mape['daily_mape_pct'].describe().round(2))

# 7-DAY FORECAST FOR EACH PIZZA

last_date = df['order_date'].max()
pizza_ids = df[['pizza_name_id', 'pizza_name_id_enc']].drop_duplicates()
forecast_rows = []

for _, row_id in pizza_ids.iterrows():

    pid = row_id['pizza_name_id']
    pid_enc = row_id['pizza_name_id_enc']

    hist = df[df['pizza_name_id_enc'] == pid_enc].copy()

    for i in range(1, 8):

        next_date = last_date + pd.Timedelta(days=i)
        row = hist.iloc[-1:].copy()
        row['order_date'] = next_date

        # Calendar features
        row['day_of_week_num'] = next_date.weekday()
        row['is_weekend'] = int(next_date.weekday() >= 5)
        row['month_num'] = next_date.month
        row['week_of_year'] = next_date.isocalendar().week
        row['day_of_month'] = next_date.day
        row['quarter'] = next_date.quarter
        row['is_month_start'] = int(next_date.is_month_start)
        row['is_month_end'] = int(next_date.is_month_end)

        # Lag features
        row['lag_1'] = hist.iloc[-1]['quantity']
        row['lag_7'] = hist.iloc[-7]['quantity']
        row['rolling_7_mean'] = hist.tail(7)['quantity'].mean()

        X_next = row[features]
        pred_qty = rf_model.predict(X_next)[0]
        pred_qty = max(0, pred_qty)

        row['quantity'] = pred_qty
        hist = pd.concat([hist, row], ignore_index=True)

        forecast_rows.append([pid, next_date, pred_qty])

forecast_df = pd.DataFrame(
    forecast_rows,
    columns=['pizza_name_id', 'forecast_date', 'predicted_quantity']
)

final_forecast = (
    forecast_df
    .groupby('pizza_name_id')['predicted_quantity']
    .sum()
    .reset_index()
)

final_forecast['predicted_quantity'] = final_forecast['predicted_quantity'].round().astype(int)

final_forecast.head()

TEST PERIOD SUM MAPE (%): 2.35
AVERAGE DAILY MAPE (%): 5.08

Daily MAPE (%):
        date  daily_mape_pct
0 2015-10-18            7.68
1 2015-10-20            5.16
2 2015-10-21            5.27
3 2015-10-22            4.98
4 2015-10-23            4.65
5 2015-10-24            5.95
6 2015-10-25            7.79
7 2015-10-27            4.18
8 2015-10-28            3.85
9 2015-10-29            4.88

Daily MAPE Summary (%):
count    71.00
mean      5.08
std       1.32
min       2.54
25%       4.20
50%       4.98
75%       5.81
max      10.21
Name: daily_mape_pct, dtype: float64


,pizza_name_id,predicted_quantity
0,bbq_ckn_l,7
1,bbq_ckn_m,7
2,bbq_ckn_s,7
3,big_meat_s,7
4,brie_carre_s,7


#### XGBRegressor

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv("train_data_without_scaling.csv")
df['order_date'] = pd.to_datetime(df['order_date'])
df = df.sort_values(['pizza_name_id', 'order_date']).reset_index(drop=True)

# ENCODE pizza_name_id
le_pizza = LabelEncoder()
df['pizza_name_id_enc'] = le_pizza.fit_transform(df['pizza_name_id'])

# Create Lag & Rolling Features
df['lag_1'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1)
df['lag_7'] = df.groupby('pizza_name_id_enc')['quantity'].shift(7)
df['lag_14'] = df.groupby('pizza_name_id_enc')['quantity'].shift(14)
df['lag_28'] = df.groupby('pizza_name_id_enc')['quantity'].shift(28)

df['rolling_7_std'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1).rolling(7).std()
df['rolling_14_mean'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1).rolling(14).mean()
df['rolling_28_mean'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1).rolling(28).mean()
df['rolling_28_std'] = df.groupby('pizza_name_id_enc')['quantity'].shift(1).rolling(28).std()
df['qty_vs_14d_avg'] = df['quantity'] / (df['rolling_14_mean'] + 1e-5)

# Lag of Same Weekday
df['lag_prev_same_weekday'] = df.groupby(['pizza_name_id_enc', 'day_of_week_num'])['quantity'].shift(1)
df = df.dropna().reset_index(drop=True)

pizza_mean = df.groupby('pizza_name_id')['quantity'].mean()
df['pizza_mean_qty'] = df['pizza_name_id'].map(pizza_mean)

# Train/Test Split
split_date = df['order_date'].quantile(0.80)
train_df = df[df['order_date'] <= split_date]
test_df = df[df['order_date'] > split_date].copy()

target = 'quantity'
drop_cols = ['quantity', 'order_date', 'pizza_name_id']
features = [col for col in df.columns if col not in drop_cols]

X_train, y_train = train_df[features], train_df[target]
X_test, y_test = test_df[features], test_df[target]

# Train XGBoost
xgb_model = XGBRegressor(
    n_estimators=800,
    max_depth=10,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

# Test Prediction
test_df['prediction'] = xgb_model.predict(X_test).clip(0)
daily_mape = test_df.groupby('order_date')[['quantity', 'prediction']].apply(
    lambda x: mean_absolute_percentage_error(x['quantity'], x['prediction']) * 100
).reset_index()
daily_mape.columns = ['date', 'daily_mape_pct']
daily_mape['daily_mape_pct'] = daily_mape['daily_mape_pct'].round(2)

# Test-period summed MAPE
actual_sum = test_df.groupby('order_date')['quantity'].sum()
pred_sum = test_df.groupby('order_date')['prediction'].sum()
mape_s = mean_absolute_percentage_error(actual_sum, pred_sum) * 100
wmape = (np.abs(test_df['quantity'] - test_df['prediction']).sum() / test_df['quantity'].sum()) * 100

print("WMAPE (%):", round(wmape, 2))
print("TEST PERIOD SUM MAPE (%):", round(mape_s, 2))
print("AVERAGE DAILY MAPE (%):", round(daily_mape['daily_mape_pct'].mean(), 2))

# 7-Day Forecast per Pizza
last_date = df['order_date'].max()
pizza_ids = df[['pizza_name_id', 'pizza_name_id_enc']].drop_duplicates()
forecast_rows = []

for _, row_id in pizza_ids.iterrows():
    pid = row_id['pizza_name_id']
    pid_enc = row_id['pizza_name_id_enc']
    hist = df[df['pizza_name_id_enc'] == pid_enc].copy()

    for i in range(1, 8):
        next_date = last_date + pd.Timedelta(days=i)
        row = hist.iloc[-1:].copy()
        row['order_date'] = next_date

        # Calendar features
        row['day_of_week_num'] = next_date.weekday()
        row['is_weekend'] = int(next_date.weekday() >= 5)
        row['month_num'] = next_date.month
        row['week_of_year'] = int(next_date.isocalendar().week)
        row['day_of_month'] = next_date.day
        row['quarter'] = next_date.quarter
        row['is_month_start'] = int(next_date.is_month_start)
        row['is_month_end'] = int(next_date.is_month_end)

        # Lag features
        row['lag_1'] = hist.iloc[-1]['quantity']
        row['lag_7'] = hist['quantity'].iloc[-7] if len(hist) >= 7 else hist.iloc[0]['quantity']
        row['lag_14'] = hist['quantity'].iloc[-14] if len(hist) >= 14 else hist.iloc[0]['quantity']
        row['lag_28'] = hist['quantity'].iloc[-28] if len(hist) >= 28 else hist.iloc[0]['quantity']

        row['rolling_7_std'] = hist['quantity'].tail(7).std()
        row['rolling_14_mean'] = hist['quantity'].tail(14).mean()
        row['rolling_28_mean'] = hist['quantity'].tail(28).mean()
        row['rolling_28_std'] = hist['quantity'].tail(28).std()
        row['qty_vs_14d_avg'] = row['lag_1'] / (row['rolling_14_mean'] + 1e-5)

        same_day_hist = hist[hist['day_of_week_num'] == next_date.weekday()]
        row['lag_prev_same_weekday'] = same_day_hist.iloc[-1]['quantity'] if len(same_day_hist) > 0 else hist.iloc[-7]['quantity']

        row['pizza_mean_qty'] = pizza_mean[pid]

        X_next = row[features]
        pred_qty = xgb_model.predict(X_next)[0]
        pred_qty = max(0, pred_qty)

        row['quantity'] = pred_qty
        hist = pd.concat([hist, row], ignore_index=True)

        forecast_rows.append([pid, next_date, pred_qty])

forecast_df = pd.DataFrame(forecast_rows, columns=['pizza_name_id', 'forecast_date', 'predicted_quantity'])

final_forecast = forecast_df.groupby('pizza_name_id')['predicted_quantity'].sum().reset_index()
final_forecast['predicted_quantity'] = final_forecast['predicted_quantity'].round().astype(int)

final_forecast.head()

WMAPE (%): 0.31
TEST PERIOD SUM MAPE (%): 0.1
AVERAGE DAILY MAPE (%): 0.29


,pizza_name_id,predicted_quantity
0,bbq_ckn_l,11
1,bbq_ckn_m,7
2,bbq_ckn_s,7
3,big_meat_s,7
4,brie_carre_s,7


#### LSTM -- 44.87%

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Embedding, Concatenate, Flatten
)
from tensorflow.keras.callbacks import EarlyStopping

# ----------------------------
# Load and preprocess data
# ----------------------------
df = pd.read_csv("train_data_without_scaling.csv")
df["order_date"] = pd.to_datetime(df["order_date"])

daily_df = (
    df.groupby(["pizza_name_id", "order_date"])
      .agg({
          "quantity": "sum",
          "is_weekend": "max",
          "day_of_week_num": "max",
          "month_num": "max"
      })
      .reset_index()
)

daily_df = daily_df.sort_values(["pizza_name_id", "order_date"]).reset_index(drop=True)
daily_df["lag_7"] = daily_df.groupby("pizza_name_id")["quantity"].shift(7)
daily_df = daily_df.dropna().reset_index(drop=True)

# Map pizza_name_id to indices
pizza_id_map = {pid: idx for idx, pid in enumerate(daily_df["pizza_name_id"].unique())}
daily_df["pizza_idx"] = daily_df["pizza_name_id"].map(pizza_id_map)
num_pizzas = len(pizza_id_map)

time_features = ["is_weekend", "day_of_week_num", "month_num", "lag_7"]
target = "quantity"
window_size = 14

# ----------------------------
# Train/test split per pizza
# ----------------------------
train_parts, test_parts = [], []
for pid in daily_df["pizza_name_id"].unique():
    pdf = daily_df[daily_df["pizza_name_id"] == pid].sort_values("order_date")
    split = int(len(pdf) * 0.8)
    train_parts.append(pdf.iloc[:split])
    test_parts.append(pdf.iloc[split:])

train_df = pd.concat(train_parts).reset_index(drop=True)
test_df = pd.concat(test_parts).reset_index(drop=True)

# ----------------------------
# Scaling
# ----------------------------
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(train_df[time_features])
X_test_scaled = scaler_X.transform(test_df[time_features])

y_train_scaled = scaler_y.fit_transform(train_df[target].values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(test_df[target].values.reshape(-1, 1))

# ----------------------------
# Create sequences per pizza (TRAIN)
# ----------------------------
Xs, ys, ps = [], [], []

for pid in train_df["pizza_idx"].unique():
    mask = train_df["pizza_idx"] == pid
    X_p = X_train_scaled[mask]
    y_p = y_train_scaled[mask]
    p_p = train_df.loc[mask, "pizza_idx"].values

    for i in range(window_size, len(X_p)):
        Xs.append(X_p[i-window_size:i])
        ys.append(y_p[i])
        ps.append(p_p[i])

X_train = np.array(Xs)
y_train = np.array(ys)
p_train = np.array(ps)

# ----------------------------
# Create sequences per pizza (TEST)
# ----------------------------
Xs_test, ys_test, ps_test = [], [], []

for pid in test_df["pizza_idx"].unique():
    mask = test_df["pizza_idx"] == pid
    X_p = X_test_scaled[mask]
    y_p = y_test_scaled[mask]
    p_p = test_df.loc[mask, "pizza_idx"].values

    for i in range(window_size, len(X_p)):
        Xs_test.append(X_p[i-window_size:i])
        ys_test.append(y_p[i])
        ps_test.append(p_p[i])

X_test = np.array(Xs_test)
y_test = np.array(ys_test)
p_test = np.array(ps_test)

# ----------------------------
# Build model
# ----------------------------
time_input = Input(shape=(window_size, X_train.shape[2]), name="time_input")
pizza_input = Input(shape=(1,), name="pizza_input")

# Pizza embedding
embed_dim = 8
pizza_embed = Embedding(input_dim=num_pizzas, output_dim=embed_dim)(pizza_input)
pizza_embed = Flatten()(pizza_embed)
pizza_embed = Dense(embed_dim, activation="relu")(pizza_embed)

# LSTM
lstm_out = LSTM(32)(time_input)

# Combine features
combined = Concatenate()([lstm_out, pizza_embed])
combined = Dropout(0.2)(combined)
output = Dense(1)(combined)

model = Model(inputs=[time_input, pizza_input], outputs=output)
model.compile(optimizer="adam", loss="mae")
model.summary()

# ----------------------------
# Train model
# ----------------------------
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model.fit(
    [X_train, p_train],
    y_train,
    validation_data=([X_test, p_test], y_test),
    epochs=40,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# ----------------------------
# Evaluate model
# ----------------------------
preds = scaler_y.inverse_transform(model.predict([X_test, p_test]))
actual = scaler_y.inverse_transform(y_test)
mape = mean_absolute_percentage_error(actual, preds) * 100
print(f"Global Daily MAPE: {mape:.2f}%")

# ----------------------------
# 7-day forecast per pizza
# ----------------------------
forecast_results = []

for pizza_id, pizza_idx in pizza_id_map.items():
    pizza_df = daily_df[daily_df["pizza_name_id"] == pizza_id].sort_values("order_date")

    if len(pizza_df) < window_size:
        continue

    last_window_raw = pizza_df[time_features].iloc[-window_size:]
    last_window_scaled = scaler_X.transform(last_window_raw)
    last_qty = pizza_df["quantity"].iloc[-window_size:].values
    last_date = pizza_df["order_date"].iloc[-1]

    future_preds = []

    for step in range(7):
        next_scaled = model.predict(
    [
        last_window_scaled.reshape(1, window_size, X_train.shape[2]),
        np.array([[pizza_idx]], dtype=np.int32)
    ],
    verbose=0
)[0, 0]

        next_qty = scaler_y.inverse_transform([[next_scaled]])[0, 0]
        future_preds.append(next_qty)

        # ---------------- Update window ----------------
        next_date = last_date + pd.Timedelta(days=1)
        # Use last 7 predicted quantities for lag
        lag_7 = future_preds[-7] if len(future_preds) >= 7 else last_qty[-7]

        new_row = pd.DataFrame([[
            int(next_date.weekday() >= 5),  # is_weekend
            next_date.weekday(),            # day_of_week_num
            next_date.month,                # month_num
            lag_7                           # lag_7
        ]], columns=time_features)

        new_row_scaled = scaler_X.transform(new_row)

        last_window_scaled = np.vstack([last_window_scaled[1:], new_row_scaled])
        last_qty = np.append(last_qty[1:], next_qty)
        last_date = next_date

    forecast_results.append({
        "pizza_name_id": pizza_id,
        "forecast_quantity_7_days": int(round(sum(future_preds)))
    })

forecast_df = pd.DataFrame(forecast_results)
forecast_df.head(20)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ pizza_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 8)      │        728 │ pizza_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_input          │ (None, 14, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 8)         │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 32)        │      4,736 │ time_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 8)         │         72 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 40)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 40)        │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         41 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,577 (21.79 KB)

 Trainable params: 5,577 (21.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 15s 18ms/step - loss: 0.0746 - val_loss: 0.0686
Epoch 2/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0674 - val_loss: 0.0723
Epoch 3/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0661 - val_loss: 0.0665
Epoch 4/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0653 - val_loss: 0.0654
Epoch 5/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0651 - val_loss: 0.0652
Epoch 6/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0647 - val_loss: 0.0652
Epoch 7/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0648 - val_loss: 0.0653
Epoch 8/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0647 - val_loss: 0.0675
Epoch 9/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.0641 - val_loss: 0.0655
Epoch 10/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0647 - val_loss: 0.0653
Epoch 11/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0642 - val_loss: 0.0654
102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Global

,pizza_name_id,forecast_quantity_7_days
0,bbq_ckn_l,19
1,bbq_ckn_m,17
2,bbq_ckn_s,12
3,big_meat_s,36
4,brie_carre_s,12
5,calabrese_l,8
6,calabrese_m,12
7,calabrese_s,6
8,cali_ckn_l,17
9,cali_ckn_m,18


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, Embedding, Concatenate
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Flatten

df = pd.read_csv("train_data_without_scaling.csv")

df["order_date"] = pd.to_datetime(df["order_date"])

daily_df = (
    df.groupby(["pizza_name_id", "order_date"])
      .agg({
          "quantity": "sum",
          "is_weekend": "max",
          "day_of_week_num": "max",
          "month_num": "max"
      })
      .reset_index()
)

daily_df = daily_df.sort_values(["pizza_name_id", "order_date"]).reset_index(drop=True)

daily_df["lag_7"] = (
    daily_df.groupby("pizza_name_id")["quantity"].shift(7)
)

daily_df = daily_df.dropna().reset_index(drop=True)

pizza_id_map = {
    pid: idx for idx, pid in enumerate(daily_df["pizza_name_id"].unique())
}

daily_df["pizza_idx"] = daily_df["pizza_name_id"].map(pizza_id_map)

num_pizzas = len(pizza_id_map)

time_features = [
    "is_weekend",
    "day_of_week_num",
    "month_num",
    "lag_7"
]

target = "quantity"

window_size = 14

train_parts = []
test_parts = []

for pid in daily_df["pizza_name_id"].unique():
    pdf = daily_df[daily_df["pizza_name_id"] == pid].sort_values("order_date")
    split = int(len(pdf) * 0.8)
    train_parts.append(pdf.iloc[:split])
    test_parts.append(pdf.iloc[split:])

train_df = pd.concat(train_parts).reset_index(drop=True)
test_df  = pd.concat(test_parts).reset_index(drop=True)

scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(train_df[time_features])
X_test_scaled  = scaler_X.transform(test_df[time_features])

y_train_scaled = scaler_y.fit_transform(
    train_df[target].values.reshape(-1, 1)
)
y_test_scaled = scaler_y.transform(
    test_df[target].values.reshape(-1, 1)
)

Xs, ys, ps = [], [], []

for pid in train_df["pizza_idx"].unique():
    mask = train_df["pizza_idx"] == pid
    X_p = X_train_scaled[mask]
    y_p = y_train_scaled[mask]
    p_p = train_df.loc[mask, "pizza_idx"].values

    for i in range(window_size, len(X_p)):
        Xs.append(X_p[i-window_size:i])
        ys.append(y_p[i])
        ps.append(p_p[i])

X_train = np.array(Xs)
y_train = np.array(ys)
p_train = np.array(ps)


Xs_test, ys_test, ps_test = [], [], []

for pid in test_df["pizza_idx"].unique():
    mask = test_df["pizza_idx"] == pid
    X_p = X_test_scaled[mask]
    y_p = y_test_scaled[mask]
    p_p = test_df.loc[mask, "pizza_idx"].values

    for i in range(window_size, len(X_p)):
        Xs_test.append(X_p[i-window_size:i])
        ys_test.append(y_p[i])
        ps_test.append(p_p[i])

X_test = np.array(Xs_test)
y_test = np.array(ys_test)
p_test = np.array(ps_test)

# Inputs
time_input = Input(shape=(window_size, X_train.shape[2]), name="time_input")
pizza_input = Input(shape=(1,), name="pizza_input")

# Pizza Embedding
embed_dim = 8
pizza_embed = Embedding(
    input_dim=num_pizzas,
    output_dim=embed_dim
)(pizza_input)

pizza_embed = Flatten()(pizza_embed)
pizza_embed = Dense(embed_dim, activation="relu")(pizza_embed)

# LSTM
lstm_out = LSTM(32)(time_input)

# Combine
combined = Concatenate()([lstm_out, pizza_embed])
combined = Dropout(0.2)(combined)
output = Dense(1)(combined)

model = Model(inputs=[time_input, pizza_input], outputs=output)

model.compile(optimizer="adam", loss="mae")

model.summary()

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model.fit(
    [X_train, p_train],
    y_train,
    validation_data=([X_test, p_test], y_test),
    epochs=40,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

preds = scaler_y.inverse_transform(
    model.predict([X_test, p_test])
)

actual = scaler_y.inverse_transform(y_test)

mape = mean_absolute_percentage_error(actual, preds) * 100
print(f"Global Daily MAPE: {mape:.2f}%")

forecast_results = []

for pizza_id, pizza_idx in pizza_id_map.items():

    pizza_df = daily_df[daily_df["pizza_name_id"] == pizza_id].copy()
    pizza_df = pizza_df.sort_values("order_date")

    if len(pizza_df) < window_size:
        continue

    last_window_raw = pizza_df[time_features].iloc[-window_size:]
    last_window_scaled = scaler_X.transform(last_window_raw)

    last_qty = pizza_df["quantity"].iloc[-window_size:].values
    last_date = pizza_df["order_date"].iloc[-1]

    future_preds = []

    for step in range(7):

        next_scaled = model.predict(
        [
        last_window_scaled.reshape(
            1, window_size, X_train.shape[2]
        ).astype(np.float32),
        np.array([[pizza_idx]], dtype=np.int32)
        ],
        verbose=0
        )[0, 0]

        next_qty = scaler_y.inverse_transform([[next_scaled]])[0, 0]
        future_preds.append(next_qty)

        # ----- Update window -----
        next_date = last_date + pd.Timedelta(days=1)

        lag_7 = last_qty[-7]

        new_row = pd.DataFrame([[
            int(next_date.weekday() >= 5),  # is_weekend
            next_date.weekday(),            # day_of_week_num
            next_date.month,                # month_num
            lag_7                           # lag_7
        ]], columns=time_features)

        new_row_scaled = scaler_X.transform(new_row)

        last_window_scaled = np.vstack([
            last_window_scaled[1:],
            new_row_scaled
        ])

        last_qty = np.append(last_qty[1:], next_qty)
        last_date = next_date

    forecast_results.append({
        "pizza_name_id": pizza_id,
        "forecast_quantity_7_days": int(round(sum(future_preds)))
    })

forecast_df = pd.DataFrame(forecast_results)

forecast_df.head()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ pizza_input         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 8)      │        728 │ pizza_input[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_input          │ (None, 14, 4)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 8)         │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 32)        │      4,736 │ time_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 8)         │         72 │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_3       │ (None, 40)        │          0 │ lstm_3[0][0],     │
│ (Concatenate)       │                   │            │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 40)        │          0 │ concatenate_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 1)         │         41 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,577 (21.79 KB)

 Trainable params: 5,577 (21.79 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.0772 - val_loss: 0.0697
Epoch 2/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - loss: 0.0663 - val_loss: 0.0666
Epoch 3/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0659 - val_loss: 0.0674
Epoch 4/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0658 - val_loss: 0.0661
Epoch 5/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 0.0647 - val_loss: 0.0658
Epoch 6/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0647 - val_loss: 0.0655
Epoch 7/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0654 - val_loss: 0.0651
Epoch 8/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0641 - val_loss: 0.0656
Epoch 9/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - loss: 0.0650 - val_loss: 0.0655
Epoch 10/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.0639 - val_loss: 0.0656
Epoch 11/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0634 - val_loss: 0.0655
Epoch 12/40
518/518 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/ste

,pizza_name_id,forecast_quantity_7_days
0,bbq_ckn_l,19
1,bbq_ckn_m,16
2,bbq_ckn_s,11
3,big_meat_s,34
4,brie_carre_s,12


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("prophet").setLevel(logging.ERROR)

# LSTM - Developments

Average Daily MAPE : 49.85%  
Average WAPE       : 41.75%  
Average Summed MAPE: 3.93%

In [ ]:
## =========================
## Imports
## =========================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


## =========================
## Load & Prepare Data
## =========================

df = pd.read_csv("train_data_without_scaling.csv")

df["order_date"] = pd.to_datetime(df["order_date"])
df = df.sort_values("order_date")

daily_df = (
    df.groupby(["pizza_name_id", "order_date"])
      .agg({
          "quantity": "sum",
          "is_weekend": "max",
          "is_month_start": "max",
          "is_month_end": "max",
          "day_of_week_num": "max",
          "month_num": "max",
          "week_of_year": "max",
          "day_of_month": "max",
          "quarter": "max"
      })
      .reset_index()
      .sort_values("order_date")
      .reset_index(drop=True)
)

forecast_results = []


## =========================
## Helper: Sequence Creator
## =========================

def create_sequences(X, window):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i - window:i])
        ys.append(X[i, 0])  # quantity (first column)
    return np.array(Xs), np.array(ys)


## =========================
## Loop per Pizza
## =========================

for pizza in daily_df["pizza_name_id"].unique():

    pizza_df = daily_df[daily_df["pizza_name_id"] == pizza].copy()

    if len(pizza_df) < 60:
        continue

    features = [
        "quantity",
        "is_weekend",
        "is_month_start",
        "is_month_end",
        "day_of_week_num",
        "month_num",
        "week_of_year",
        "day_of_month",
        "quarter"
    ]

    X = pizza_df[features].values

    train_size = int(len(X) * 0.8)

    X_train_raw = X[:train_size]
    X_test_raw  = X[train_size:]

    ## =========================
    ## Scaling (ONE scaler only)
    ## =========================

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_raw)
    X_test_scaled  = scaler.transform(X_test_raw)

    ## =========================
    ## Create Sequences
    ## =========================

    window_size = 21

    X_train, y_train = create_sequences(X_train_scaled, window_size)

    X_test, y_test = create_sequences(
        np.vstack([X_train_scaled[-window_size:], X_test_scaled]),
        window_size
    )

    ## =========================
    ## Model
    ## =========================

    model = Sequential([
        Input(shape=(window_size, X_train.shape[2])),
        LSTM(32),
        Dropout(0.3),
        Dense(1)
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.Huber()
    )

    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        epochs=40,
        batch_size=16,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        verbose=0
    )

    ## =========================
    ## Predictions & Metrics
    ## =========================

    preds_scaled = model.predict(X_test, verbose=0)

    # inverse transform quantity only
    dummy = np.zeros((len(preds_scaled), X_train.shape[2]))
    dummy[:, 0] = preds_scaled.flatten()
    preds = scaler.inverse_transform(dummy)[:, 0]

    dummy[:, 0] = y_test
    actual = scaler.inverse_transform(dummy)[:, 0]

    mape = mean_absolute_percentage_error(actual, preds) * 100
    wape = np.sum(np.abs(actual - preds)) / np.sum(actual) * 100

    actual_sum = actual.sum()
    pred_sum = preds.sum()
    mape_s = abs(actual_sum - pred_sum) / actual_sum * 100

    ## =========================
    ## 7-Day Forecast
    ## =========================

    last_window = np.vstack([X_train_scaled, X_test_scaled])[-window_size:]
    last_date = pizza_df["order_date"].iloc[-1]

    future_preds = []

    for _ in range(7):

        next_scaled_qty = model.predict(
            last_window.reshape(1, window_size, -1),
            verbose=0
        )[0, 0]

        dummy = np.zeros((1, X_train.shape[2]))
        dummy[0, 0] = next_scaled_qty
        next_qty = scaler.inverse_transform(dummy)[0, 0]

        future_preds.append(next_qty)

        next_date = last_date + pd.Timedelta(days=1)

        new_row = np.array([[
            next_qty,
            int(next_date.weekday() >= 5),
            int(next_date.is_month_start),
            int(next_date.is_month_end),
            next_date.weekday(),
            next_date.month,
            int(next_date.isocalendar().week),
            next_date.day,
            next_date.quarter
        ]])

        new_row_scaled = scaler.transform(new_row)

        last_window = np.vstack([last_window[1:], new_row_scaled])
        last_date = next_date

    forecast_results.append({
        "pizza_name_id": pizza,
        "forecast_quantity_7_days": int(round(sum(future_preds))),
        "MAPE_daily": round(mape, 2),
        "WAPE": round(wape, 2),
        "MAPE_summed": round(mape_s, 2)
    })


## =========================
## Final Results
## =========================

forecast_df = pd.DataFrame(forecast_results)

print(f"Average Daily MAPE : {forecast_df['MAPE_daily'].mean():.2f}%")
print(f"Average WAPE       : {forecast_df['WAPE'].mean():.2f}%")
print(f"Average Summed MAPE: {forecast_df['MAPE_summed'].mean():.2f}%")

forecast_df

Average Daily MAPE : 49.85%
Average WAPE       : 41.75%
Average Summed MAPE: 3.93%


,pizza_name_id,forecast_quantity_7_days,MAPE_daily,WAPE,MAPE_summed
0,bbq_ckn_l,22,62.66,42.60,1.71
1,calabrese_m,13,57.97,45.19,1.57
2,thai_ckn_m,15,52.21,44.56,2.43
3,mediterraneo_m,9,46.61,43.36,0.46
4,prsc_argla_m,10,43.47,36.23,8.39
...,...,...,...,...,...
85,veggie_veg_m,10,59.92,42.66,7.20
86,soppressata_s,10,42.55,41.19,7.16
87,mediterraneo_s,12,42.27,36.10,4.37
88,ital_veggie_l,7,27.94,31.55,3.80


Average Daily MAPE : 50.32%  
Average WAPE       : 42.00%  
Average Summed MAPE: 4.08%

In [ ]:
## =========================
## Imports
## =========================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


## =========================
## Load & Prepare Data
## =========================

df = pd.read_csv("train_data_without_scaling.csv")

df["order_date"] = pd.to_datetime(df["order_date"])
df = df.sort_values("order_date")

daily_df = (
    df.groupby(["pizza_name_id", "order_date"])
      .agg({
          "quantity": "sum",
          "is_weekend": "max",
          "is_month_start": "max",
          "is_month_end": "max",
          "day_of_week_num": "max",
          "month_num": "max",
          "week_of_year": "max",
          "day_of_month": "max",
          "quarter": "max"
      })
      .reset_index()
      .sort_values("order_date")
      .reset_index(drop=True)
)

forecast_results = []


## =========================
## Helper: Sequence Creator
## =========================

def create_sequences(X, window):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i - window:i])
        ys.append(X[i, 0])  # quantity (first column)
    return np.array(Xs), np.array(ys)


## =========================
## Loop per Pizza
## =========================

for pizza in daily_df["pizza_name_id"].unique():

    pizza_df = daily_df[daily_df["pizza_name_id"] == pizza].copy()

    if len(pizza_df) < 60:
        continue

    features = [
        "quantity",
        "is_weekend",
        "is_month_start",
        "is_month_end",
        "day_of_week_num",
        "month_num",
        "week_of_year",
        "day_of_month",
        "quarter"
    ]

    X = pizza_df[features].values

    train_size = int(len(X) * 0.8)

    X_train_raw = X[:train_size]
    X_test_raw  = X[train_size:]

    ## =========================
    ## Scaling (ONE scaler only)
    ## =========================

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_raw)
    X_test_scaled  = scaler.transform(X_test_raw)

    ## =========================
    ## Create Sequences
    ## =========================

    window_size = 21

    X_train, y_train = create_sequences(X_train_scaled, window_size)

    X_test, y_test = create_sequences(
        np.vstack([X_train_scaled[-window_size:], X_test_scaled]),
        window_size
    )

    ## =========================
    ## Model
    ## =========================

    model = Sequential([
        Input(shape=(window_size, X_train.shape[2])),
        LSTM(64),
        Dropout(0.3),
        Dense(1)
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.Huber()
    )

    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        epochs=40,
        batch_size=16,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        verbose=0
    )

    ## =========================
    ## Predictions & Metrics
    ## =========================

    preds_scaled = model.predict(X_test, verbose=0)

    # inverse transform quantity only
    dummy = np.zeros((len(preds_scaled), X_train.shape[2]))
    dummy[:, 0] = preds_scaled.flatten()
    preds = scaler.inverse_transform(dummy)[:, 0]

    dummy[:, 0] = y_test
    actual = scaler.inverse_transform(dummy)[:, 0]

    mape = mean_absolute_percentage_error(actual, preds) * 100
    wape = np.sum(np.abs(actual - preds)) / np.sum(actual) * 100

    actual_sum = actual.sum()
    pred_sum = preds.sum()
    mape_s = abs(actual_sum - pred_sum) / actual_sum * 100

    ## =========================
    ## 7-Day Forecast
    ## =========================

    last_window = np.vstack([X_train_scaled, X_test_scaled])[-window_size:]
    last_date = pizza_df["order_date"].iloc[-1]

    future_preds = []

    for _ in range(7):

        next_scaled_qty = model.predict(
            last_window.reshape(1, window_size, -1),
            verbose=0
        )[0, 0]

        dummy = np.zeros((1, X_train.shape[2]))
        dummy[0, 0] = next_scaled_qty
        next_qty = scaler.inverse_transform(dummy)[0, 0]

        future_preds.append(next_qty)

        next_date = last_date + pd.Timedelta(days=1)

        new_row = np.array([[
            next_qty,
            int(next_date.weekday() >= 5),
            int(next_date.is_month_start),
            int(next_date.is_month_end),
            next_date.weekday(),
            next_date.month,
            int(next_date.isocalendar().week),
            next_date.day,
            next_date.quarter
        ]])

        new_row_scaled = scaler.transform(new_row)

        last_window = np.vstack([last_window[1:], new_row_scaled])
        last_date = next_date

    forecast_results.append({
        "pizza_name_id": pizza,
        "forecast_quantity_7_days": int(round(sum(future_preds))),
        "MAPE_daily": round(mape, 2),
        "WAPE": round(wape, 2),
        "MAPE_summed": round(mape_s, 2)
    })


## =========================
## Final Results
## =========================

forecast_df = pd.DataFrame(forecast_results)

print(f"Average Daily MAPE : {forecast_df['MAPE_daily'].mean():.2f}%")
print(f"Average WAPE       : {forecast_df['WAPE'].mean():.2f}%")
print(f"Average Summed MAPE: {forecast_df['MAPE_summed'].mean():.2f}%")

forecast_df

Average Daily MAPE : 50.32%
Average WAPE       : 42.00%
Average Summed MAPE: 4.08%


,pizza_name_id,forecast_quantity_7_days,MAPE_daily,WAPE,MAPE_summed
0,bbq_ckn_l,19,66.14,43.97,2.25
1,calabrese_m,13,62.03,46.89,2.59
2,thai_ckn_m,12,51.39,43.81,2.61
3,mediterraneo_m,9,50.32,44.75,4.33
4,prsc_argla_m,13,45.63,37.07,5.68
...,...,...,...,...,...
85,veggie_veg_m,14,45.20,38.00,11.18
86,soppressata_s,7,23.36,30.62,16.71
87,mediterraneo_s,12,40.18,37.84,5.08
88,ital_veggie_l,9,33.01,34.65,2.78


Average Daily MAPE : 50.34%  
Average WAPE       : 42.10%  
Average Summed MAPE: 3.93%

In [ ]:
## =========================
## Imports
## =========================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


## =========================
## Load & Prepare Data
## =========================

df = pd.read_csv("train_data_without_scaling.csv")

df["order_date"] = pd.to_datetime(df["order_date"])
df = df.sort_values("order_date")

daily_df = (
    df.groupby(["pizza_name_id", "order_date"])
      .agg({
          "quantity": "sum",
          "is_weekend": "max",
          "is_month_start": "max",
          "is_month_end": "max",
          "day_of_week_num": "max",
          "month_num": "max",
          "week_of_year": "max",
          "day_of_month": "max",
          "quarter": "max"
      })
      .reset_index()
      .sort_values("order_date")
      .reset_index(drop=True)
)

forecast_results = []


## =========================
## Helper: Sequence Creator
## =========================

def create_sequences(X, window):
    Xs, ys = [], []
    for i in range(window, len(X)):
        Xs.append(X[i - window:i])
        ys.append(X[i, 0])  # quantity (first column)
    return np.array(Xs), np.array(ys)


## =========================
## Loop per Pizza
## =========================

for pizza in daily_df["pizza_name_id"].unique():

    pizza_df = daily_df[daily_df["pizza_name_id"] == pizza].copy()

    if len(pizza_df) < 60:
        continue

    features = [
        "quantity",
        "is_weekend",
        "is_month_start",
        "is_month_end",
        "day_of_week_num",
        "month_num",
        "week_of_year",
        "day_of_month",
        "quarter"
    ]

    X = pizza_df[features].values

    train_size = int(len(X) * 0.8)

    X_train_raw = X[:train_size]
    X_test_raw  = X[train_size:]

    ## =========================
    ## Scaling (ONE scaler only)
    ## =========================

    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train_raw)
    X_test_scaled  = scaler.transform(X_test_raw)

    ## =========================
    ## Create Sequences
    ## =========================

    window_size = 21

    X_train, y_train = create_sequences(X_train_scaled, window_size)

    X_test, y_test = create_sequences(
        np.vstack([X_train_scaled[-window_size:], X_test_scaled]),
        window_size
    )

    ## =========================
    ## Model
    ## =========================

    model = Sequential([
        Input(shape=(window_size, X_train.shape[2])),
        LSTM(32),
        Dropout(0.3),
        Dense(1)
    ])

    model.compile(
        optimizer="adam",
        loss=tf.keras.losses.Huber()
    )

    early_stop = EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )

    model.fit(
        X_train, y_train,
        epochs=40,
        batch_size=16,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        verbose=0
    )

    ## =========================
    ## Predictions & Metrics
    ## =========================

    preds_scaled = model.predict(X_test, verbose=0)

    # inverse transform quantity only
    dummy = np.zeros((len(preds_scaled), X_train.shape[2]))
    dummy[:, 0] = preds_scaled.flatten()
    preds = scaler.inverse_transform(dummy)[:, 0]

    dummy[:, 0] = y_test
    actual = scaler.inverse_transform(dummy)[:, 0]

    mape = mean_absolute_percentage_error(actual, preds) * 100
    wape = np.sum(np.abs(actual - preds)) / np.sum(actual) * 100

    actual_sum = actual.sum()
    pred_sum = preds.sum()
    mape_s = abs(actual_sum - pred_sum) / actual_sum * 100

    ## =========================
    ## 7-Day Forecast
    ## =========================

    last_window = np.vstack([X_train_scaled, X_test_scaled])[-window_size:]
    last_date = pizza_df["order_date"].iloc[-1]

    future_preds = []

    for _ in range(7):

        next_scaled_qty = model.predict(
            last_window.reshape(1, window_size, -1),
            verbose=0
        )[0, 0]

        dummy = np.zeros((1, X_train.shape[2]))
        dummy[0, 0] = next_scaled_qty
        next_qty = scaler.inverse_transform(dummy)[0, 0]

        future_preds.append(next_qty)

        next_date = last_date + pd.Timedelta(days=1)

        new_row = np.array([[
            next_qty,
            int(next_date.weekday() >= 5),
            int(next_date.is_month_start),
            int(next_date.is_month_end),
            next_date.weekday(),
            next_date.month,
            int(next_date.isocalendar().week),
            next_date.day,
            next_date.quarter
        ]])

        new_row_scaled = scaler.transform(new_row)

        last_window = np.vstack([last_window[1:], new_row_scaled])
        last_date = next_date

    forecast_results.append({
        "pizza_name_id": pizza,
        "forecast_quantity_7_days": int(round(sum(future_preds))),
        "MAPE_daily": round(mape, 2),
        "WAPE": round(wape, 2),
        "MAPE_summed": round(mape_s, 2)
    })


## =========================
## Final Results
## =========================

forecast_df = pd.DataFrame(forecast_results)

print(f"Average Daily MAPE : {forecast_df['MAPE_daily'].mean():.2f}%")
print(f"Average WAPE       : {forecast_df['WAPE'].mean():.2f}%")
print(f"Average Summed MAPE: {forecast_df['MAPE_summed'].mean():.2f}%")

forecast_df

Average Daily MAPE : 50.34%
Average WAPE       : 42.10%
Average Summed MAPE: 3.93%


,pizza_name_id,forecast_quantity_7_days,MAPE_daily,WAPE,MAPE_summed
0,bbq_ckn_l,20,62.08,43.07,2.65
1,calabrese_m,9,57.70,45.04,2.15
2,thai_ckn_m,14,52.81,45.16,2.19
3,mediterraneo_m,9,47.50,43.29,1.76
4,prsc_argla_m,13,38.96,35.54,15.18
...,...,...,...,...,...
85,veggie_veg_m,18,48.29,38.56,6.66
86,soppressata_s,13,32.40,34.83,3.14
87,mediterraneo_s,16,41.10,37.18,1.12
88,ital_veggie_l,7,29.91,32.39,0.50
